In [ ]:
import websocket, json
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

# Define a threshold for buy/sell signals
threshold = 0.01

# Define a sequence length to match the model
sequence_length = 15

# Initialize data to store candle information
candle_data = []

def load_lstm_model():
    data = pd.read_csv('/content/Final ADAUSDT Data.csv')  # Load your data file
    data = data[['close']]
    scaler = MinMaxScaler()
    data['close'] = scaler.fit_transform(data[['close']])

    # Convert the dataset into sequences for time series forecasting
    def create_sequences(data, seq_length):
        sequences = []
        for i in range(len(data) - seq_length):
            sequence = data[i:i + seq_length]
            sequences.append(sequence)
        return np.array(sequences)

    sequences = create_sequences(data, sequence_length)
    train_size = int(len(sequences) * 0.8)
    train_data = sequences[:train_size]

    # Split the data into features (X) and target (y)
    X_train = train_data[:, :-1]
    y_train = train_data[:, -1]

    # Build the LSTM model
    model = Sequential()
    model.add(LSTM(units=50, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dense(units=1))
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Train the model
    model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)

    return model, scaler

# Load the LSTM model and scaler
model, scaler = load_lstm_model()

In [ ]:
import websocket, json
import pandas as pd

# Define a threshold for buy/sell signals
threshold = 0.01

# Define a sequence length to match the model
sequence_length = 14

# Initialize data to store candle information
candle_data = []

# Define scaler as a global variable
scaler = None

# Opening connection to the WebSocket
def on_open(ws):
    print("Opened Connection")

# Closing connection to the WebSocket
def on_close(ws, close_status_code, close_msg):
    print("Closed Connection")

# Executes every time a message is received from the WebSocket
def on_message(ws, message):
    json_message = json.loads(message)

    # Extract candle data
    candle = json_message['k']

    # Check if it's a candle close event
    if candle['x']:
        process_candle(candle)

# Function to prepare new data and generate signals
def process_candle(candle):
    global scaler  # Use the global scaler variable

    # Append the candle data
    candle_data.append(candle)

    # Keep the sequence length (last 14 candles)
    if len(candle_data) > sequence_length:
        candle_data.pop(0)

    # Check if we have enough data for prediction
    if len(candle_data) == sequence_length:
        # Extract 'close' prices from the candle data
        close_prices = [float(candle['c']) for candle in candle_data]

        # Prepare new data
        new_data = pd.DataFrame({'close': close_prices})

        # Normalize the new data using the same scaler used for training
        new_data['close'] = scaler.transform(new_data[['close']])

        # Create a sequence for prediction
        new_sequence = new_data.values
        new_sequence = new_sequence.reshape(1, sequence_length, 1)

        # Make a prediction for the next candle's close price
        predicted_normalized_close = model.predict(new_sequence)

        # Inverse transform the prediction to get the actual close price
        predicted_close = scaler.inverse_transform(predicted_normalized_close)

        # Generate buy/sell signal
        previous_close = candle_data[-2]['c']
        price_change = (predicted_close - previous_close) / previous_close

        if price_change > threshold:
            signal = 1  # Buy signal
        elif price_change < -threshold:
            signal = 0  # Sell signal
        else:
            signal = -1  # No action (optional)

        print("Signal:", signal)

# WebSocket stream
SOCKET = "wss://fstream.binance.com/ws/adausdt_perpetual@continuousKline_1m"

# Configure WebSocket connection and run
ws = websocket.WebSocketApp(SOCKET, on_open=on_open, on_close=on_close, on_message=on_message)
ws.run_forever()

Opened Connection
Signal: -1
Signal: -1
Signal: 0
Signal: 0
Signal: 1
Signal: -1
Signal: -1
Closed Connection
